# **Tarea 3**

## **CC6204 - Deep Learning**

- **Profesor:** Iván Sipiran
- **Ayudantes:** Camila Figueroa Acevedo, Gustavo Santelices, Sofia Capibara Chávez Bastidas, Victor Faraggi V.
- **Estudiante:** Sebastián Sanhueza

En esta tarea van a crear una red neuronal que clasifique mensajes como spam o no spam. Lo primero es descargar la data:

In [1]:
!wget https://www.ivan-sipiran.com/downloads/spam.csv

--2023-11-20 02:25:28--  https://www.ivan-sipiran.com/downloads/spam.csv
Resolving www.ivan-sipiran.com (www.ivan-sipiran.com)... 66.96.149.31
Connecting to www.ivan-sipiran.com (www.ivan-sipiran.com)|66.96.149.31|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 471781 (461K)
Saving to: ‘spam.csv’

spam.csv            100%[===================>] 460.72K  1.62MB/s    in 0.3s    

2023-11-20 02:25:29 (1.62 MB/s) - ‘spam.csv’ saved [471781/471781]



Los datos vienen en un archivo CSV que contiene dos columnas "text" y "label". La columna "text" contiene el texto del mensaje y la columna "label" contiene las etiquetas "ham" y "spam". Un mensaje "ham" es un mensaje que no se considera spam.

# Tarea
El objetivo de la tarea es crear una red neuronal que clasifique los datos entregados. Para lograr esto debes:



*   Implementar el pre-procesamiento de los datos que creas necesario.
*   Particionar los datos en 70% entrenamiento, 10% validación y 20% test.
*   Usa los datos de entrenamiento y valiadación para tus experimentos y sólo usa el conjunto de test para reportar el resultado final.

Para el diseño de la red neuronal puedes usar una red neuronal recurrente o una red basada en transformers. El objetivo de la tarea no es obtener el performance ultra máximo, sino entender qué decisiones de diseño afectan la solución de un problema como este. Lo que si es necesario (como siempre) es que discutas los resultados y decisiones realizadas.



## Pre-procesamiento de datos

### Análisis de los datos

In [2]:
# Librerías
import pandas as pd
import numpy as np
import seaborn as sns
from string import punctuation
from collections import Counter
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

In [3]:
# Seed para numpy
np_seed = 42
np.random.seed(np_seed)

# Seed para torch
torch_seed = 42
torch.manual_seed(torch_seed)

# Seed para torch en CUDA (si se está utilizando)
if torch.cuda.is_available():
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.cuda.manual_seed_all(torch_seed)


In [4]:
# Carga de la data
df=pd.read_csv("spam.csv")
df.head(10)

,text,label
0,"Go until jurong point, crazy.. Available only ...",ham
1,Ok lar... Joking wif u oni...,ham
2,Free entry in 2 a wkly comp to win FA Cup fina...,spam
3,U dun say so early hor... U c already then say...,ham
4,"Nah I don't think he goes to usf, he lives aro...",ham
5,FreeMsg Hey there darling it's been 3 week's n...,spam
6,Even my brother is not like to speak with me. ...,ham
7,As per your request 'Melle Melle (Oru Minnamin...,ham
8,WINNER!! As a valued network customer you have...,spam
9,Had your mobile 11 months or more? U R entitle...,spam


Inicialmente, se eliminaron los espacios al inicio y final de las cadenas de texto. Este paso se realizó porque durante el análisis de datos duplicados se encontraron registros idénticos que se diferenciaban únicamente por estos espacios adicionales. Posteriormente, se llevó a cabo un análisis de las dimensiones del conjunto de datos, incluyendo el recuento total de registros y variables originales. Además, se realizó una búsqueda de valores nulos en los datos, ya que la presencia de estos podría llevar a errores en el modelo que se pretende implementar.

In [5]:
# Eliminamos los espacios del inicio y del final de cada string
df['text']=df['text'].str.strip()
df['label']=df['label'].str.strip()

# Análisis de las dimensiones del DataFrame
n_rows=df.shape[0]
n_columns=df.shape[1]
print(f"Cantidad total de registros: {n_rows}")
print(f"\nCantidad total de features: {n_columns}")
print(f"\nNombre de las columnas: {list(df.columns)}")

# Análisis de valores nulos
n_nan_text=df.isna().sum()['text']
n_nan_label=df.isna().sum()['label']
print(f"\nCantidad de valores nulos en cada feature: 'text': {n_nan_text}, 'label': {n_nan_label}")
print("\nRegistros con valores nulos:")
df[df.isna().any(axis=1)]

Cantidad total de registros: 5572

Cantidad total de features: 2

Nombre de las columnas: ['text', 'label']

Cantidad de valores nulos en cada feature: 'text': 1, 'label': 1

Registros con valores nulos:


,text,label
2115,Well I wasn't available as I washob nobbing wi...,NaN
3035,NaN,-) ok. I feel like john lennon.


Tras el análisis previo, se constató que el conjunto de datos original contiene un total de 5572 registros, con dos variables: `text` y `spam`. Además, se identificó la presencia de datos nulos en dos registros, uno en cada variable. Estos registros serán eliminados durante la etapa de limpieza del dataset.

En una segunda etapa, se llevó a cabo una búsqueda de datos duplicados o repetidos más de una vez en el dataset original. La presencia de valores repetidos podría distorsionar los resultados de análisis, impactar la eficiencia de los modelos de machine learning, introducir sesgos y crear inconsistencias en la interpretación de los datos. Identificar y eliminar estos valores duplicados asegura la fiabilidad y calidad de las conclusiones extraídas del conjunto de datos, garantizando así la integridad de cualquier análisis posterior.

In [6]:
# Análisis de valores duplicados

# Creación de un DataFrame que agrega una columna 'count' que realiza el conteo de las veces que aparece
# cada registro dentro del DataFrame original
repeated_rows=df.groupby(df.columns.tolist()).size().reset_index(name='count')
print(f"\nResumen de la cantidad de registros con una misma cantidad de repeticiones: \n{repeated_rows['count'].value_counts()}")
print(f"\nCantidad total de registros únicos con más de una aparición en el DataFrame: {repeated_rows[repeated_rows['count']!=1].shape[0]}")


Resumen de la cantidad de registros con una misma cantidad de repeticiones: 
1     4861
2      215
3       63
4        8
6        1
12       1
10       1
30       1
Name: count, dtype: int64

Cantidad total de registros únicos con más de una aparición en el DataFrame: 290


Los resultados del análisis revelan la presencia de 290 registros únicos que se repiten más de una vez en el dataset original. Es notable que la mayoría de los datos repetidos se encuentran duplicados, aunque se identificaron tres registros únicos que se repiten más de 10 veces, destacando uno con una frecuencia de 30 repeticiones en el conjunto de datos inicial. Esta observación resalta la necesidad de realizar una minuciosa eliminación de estas repeticiones, ya que pueden impactar negativamente en el rendimiento de los modelos de machine learning, afectando la precisión y confiabilidad de los resultados de clasificación que se buscan obtener.

In [7]:
# Análisis de registros erróneos

# Se identificacarán registros que tengan algún valor de 'label' distinto a 'ham' y 'spam'
reg_errors=df[(df['label']!="ham")&(df['label']!="spam")]
print(f"Cantidad de registros con el valor de 'label' distinto a 'ham' y 'spam': {reg_errors.shape[0]}")

# Visualización de todos los valores únicos en la variable 'label'
conteo_valores_unicos=df['label'].value_counts()
print("\nValores únicos en la variable 'label':")
# Acceder a todos los valores únicos y sus conteos en un bucle
for valor, conteo in conteo_valores_unicos.items():
    print(f"Valor: {valor}, Conteo: {conteo}")

Cantidad de registros con el valor de 'label' distinto a 'ham' y 'spam': 209

Valores únicos en la variable 'label':
Valor: ham, Conteo: 4617
Valor: spam, Conteo: 746
Valor: #&gt, Conteo: 136
Valor: DECIMAL&gt, Conteo: 13
Valor: _, Conteo: 4
Valor: URL&gt, Conteo: 3
Valor: :(, Conteo: 3
Valor: successful day., Conteo: 3
Valor: HAVE A NICE SLEEP..SWEET DREAMS.., Conteo: 2
Valor: -), Conteo: 2
Valor: it kills me that u don't care enough to stop me..., Conteo: 2
Valor: Take care, Conteo: 2
Valor: sweet dreams, Conteo: 2
Valor: wish U Merry Xmas..., Conteo: 2
Valor: TIME&gt, Conteo: 2
Valor: they wer askd 2 sit in an aeroplane. Aftr they sat they wer told dat the plane ws made by their students. Dey all hurried out of d plane.. Bt only 1 didnt move... He said:\if it is made by my students, Conteo: 2
Valor: ), Conteo: 2
Valor: Life is empty without frnds.. So Alwys Be In Touch. Good night &amp, Conteo: 2
Valor: present number..:) By Rajitha Raj (Ranju), Conteo: 1
Valor: ) None? People hate 

A partir del análisis anterior, se identificaron 209 registros que contienen etiquetas distintas a las esperadas en la variable `label`. Además, se observó que en muchos casos estas etiquetas erróneas fueron reemplazadas por otros textos o símbolos, lo que claramente representa una fuente de ruido en los datos. Esta inconsistencia puede tener un impacto negativo en el rendimiento de nuestro modelo, ya que introduce información incorrecta o inesperada que puede dificultar el proceso de aprendizaje y la precisión de las predicciones.

### Limpieza de los datos

En el siguiente paso, se llevará a cabo una limpieza de los datos originales con el objetivo de eliminar los errores y eliminar las fuentes de ruido detectadas previamente. Es crucial seguir un orden específico durante este proceso, dado que realizarlo de manera aleatoria podría resultar en la pérdida de registros únicos dentro de los datos, comprometiendo así la integridad y calidad del conjunto de datos.

Primero, se eliminarán los datos con errores en la variable `label`, es decir, aquellos registros que contienen valores distintos a `ham` y `spam`, las únicas etiquetas permitidas en el DataFrame.

In [8]:
# Eliminación de datos erróneos
filas_a_eliminar=reg_errors.index
df_clean = df.drop(filas_a_eliminar)

En segundo término, se eliminarán todos los datos nulos que se encuentren en el DataFrame.

In [9]:
# Eliminación de los datos nulos
df_clean.dropna(inplace=True)

En el tercer paso, se eliminarán todos los registros duplicados en el DataFrame, manteniendo únicamente un registro único de cada uno. Esta acción tiene como objetivo eliminar cualquier ruido o efecto negativo que la información repetida y redundante pueda provocar en el análisis.

In [10]:
# Eliminación de datos duplicados
df_clean.drop_duplicates(inplace=True)

Para asegurarnos de que los cambios se aplicaron corectamente se realizará el mismo análisis que a los datos originales.

In [11]:
df_clean.reset_index(drop=True, inplace=True)
df_clean.head(10)

,text,label
0,"Go until jurong point, crazy.. Available only ...",ham
1,Ok lar... Joking wif u oni...,ham
2,Free entry in 2 a wkly comp to win FA Cup fina...,spam
3,U dun say so early hor... U c already then say...,ham
4,"Nah I don't think he goes to usf, he lives aro...",ham
5,FreeMsg Hey there darling it's been 3 week's n...,spam
6,Even my brother is not like to speak with me. ...,ham
7,As per your request 'Melle Melle (Oru Minnamin...,ham
8,WINNER!! As a valued network customer you have...,spam
9,Had your mobile 11 months or more? U R entitle...,spam


In [12]:
# Análisis de las dimensiones del DataFrame
n_rows=df_clean.shape[0]
n_columns=df_clean.shape[1]
print(f"Cantidad total de registros: {n_rows}")
print(f"\nCantidad total de features: {n_columns}")
print(f"\nNombre de las columnas: {list(df_clean.columns)}")

# Análisis de valores nulos
n_nan_text=df_clean.isna().sum()['text']
n_nan_label=df_clean.isna().sum()['label']
print(f"\nCantidad de valores nulos en cada feature: 'text': {n_nan_text}, 'label': {n_nan_label}")
print("\nRegistros con valores nulos:")
df_clean[df_clean.isna().any(axis=1)]

Cantidad total de registros: 4966

Cantidad total de features: 2

Nombre de las columnas: ['text', 'label']

Cantidad de valores nulos en cada feature: 'text': 0, 'label': 0

Registros con valores nulos:


,text,label


In [13]:
# Análisis de valores duplicados

# Creación de un DataFrame que agrega una columna 'count' que realiza el conteo de las veces que aparece
# cada registro dentro del DataFrame original
repeated_rows=df_clean.groupby(df_clean.columns.tolist()).size().reset_index(name='count')
print(f"\nResumen de la cantidad de registros con una misma cantidad de repeticiones: \n{repeated_rows['count'].value_counts()}")
print(f"\nCantidad total de registros únicos con más de una aparición en el DataFrame: {repeated_rows[repeated_rows['count']!=1].shape[0]}")


Resumen de la cantidad de registros con una misma cantidad de repeticiones: 
1    4966
Name: count, dtype: int64

Cantidad total de registros únicos con más de una aparición en el DataFrame: 0


In [14]:
# Análisis de registros erróneos

# Se identificacarán registros que tengan algún valor de 'label' distinto a 'ham' y 'spam'
reg_errors=df_clean[(df_clean['label']!="ham")&(df_clean['label']!="spam")]
print(f"Cantidad de registros con el valor de 'label' distinto a 'ham' y 'spam': {reg_errors.shape[0]}")

# Visualización de todos los valores únicos en la variable 'label'
conteo_valores_unicos=df_clean['label'].value_counts()
print("\nValores únicos en la variable 'label':")
# Acceder a todos los valores únicos y sus conteos en un bucle
for valor, conteo in conteo_valores_unicos.items():
    print(f"Valor: {valor}, Conteo: {conteo}")

Cantidad de registros con el valor de 'label' distinto a 'ham' y 'spam': 0

Valores únicos en la variable 'label':
Valor: ham, Conteo: 4325
Valor: spam, Conteo: 641


### Eliminación de signos de puntuación y división de texto en palabras individuales

Luego de la limpieza de los datos se realizará un procesamiento del texto contenido en los mensajes y así convertirlo en una representación idónea para ser procesada mediante una red neuronal.

En una primera etapa, se procederá a dividir el conjunto de datos en dos arreglos distintos: `messages` y `labels`, que contendrán, respectivamente, los textos de los mensajes y las etiquetas asociadas a cada mensaje. Posteriormente, los textos presentes en estos arreglos se combinarán para formar un solo string en cada uno, consolidando así toda la información contenida en cada arreglo.

In [15]:
# Separación de los datos en messages y labels
messages=[]
labels=[]

for i, label in enumerate(df_clean['label']):
    messages.append(df_clean['text'][i])
    labels.append(label)

messages=np.asarray(messages)
labels=np.asarray(labels)

# Unión de los textos en cada arreglo para formar un solo string
messages='\n'.join(messages)
labels='\n'.join(labels)

In [16]:
# Visualización de parte de los strings de cada arreglo
print(f"{messages[:200]}")
print(f"\n{labels[:6]}")

Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...
Ok lar... Joking wif u oni...
Free entry in 2 a wkly comp to win FA Cup final tkts 21st 

ham
ha


Después, se llevó a cabo la eliminación de los signos de puntuación, dado que no son relevantes para nuestro análisis. Además, se procedió a segmentar el texto en palabras individuales, facilitando así el procesamiento y análisis de cada término de manera independiente.

In [17]:
print(punctuation) # Visualización de los signos de puntuación considerados

messages = messages.lower() # Transformación del texto a solo letras minúsculas

all_text = ''.join([c for c in messages if c not in punctuation]) # Se une el texto sin los signos de puntuación

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [18]:
# Eliminación de los saltos de línea
messages_split = all_text.split('\n')
all_text = ' '.join(messages_split)

# Separación del texto en palabras individuales
words = all_text.split()
print(words[:100])

['go', 'until', 'jurong', 'point', 'crazy', 'available', 'only', 'in', 'bugis', 'n', 'great', 'world', 'la', 'e', 'buffet', 'cine', 'there', 'got', 'amore', 'wat', 'ok', 'lar', 'joking', 'wif', 'u', 'oni', 'free', 'entry', 'in', '2', 'a', 'wkly', 'comp', 'to', 'win', 'fa', 'cup', 'final', 'tkts', '21st', 'may', '2005', 'text', 'fa', 'to', '87121', 'to', 'receive', 'entry', 'questionstd', 'txt', 'ratetcs', 'apply', '08452810075over18s', 'u', 'dun', 'say', 'so', 'early', 'hor', 'u', 'c', 'already', 'then', 'say', 'nah', 'i', 'dont', 'think', 'he', 'goes', 'to', 'usf', 'he', 'lives', 'around', 'here', 'though', 'freemsg', 'hey', 'there', 'darling', 'its', 'been', '3', 'weeks', 'now', 'and', 'no', 'word', 'back', 'id', 'like', 'some', 'fun', 'you', 'up', 'for', 'it', 'still']


### Codificación de palabras

A continuación se representará a cada palabra con un número entero único que representará su índice. Para lograr esto, crearemos un diccionario que establezca la correspondencia entre palabras y números, permitiéndonos transformar las palabras del texto en valores numéricos de manera eficiente.

In [19]:
counts = Counter(words) # Construye un diccionario de palabras. Las claves son las palabras y los valores son la frecuencia
vocab = sorted(counts, key=counts.get, reverse=True) # Ordena la palabras por frecuencia
vocab_to_int = {word: ii for ii, word in enumerate(vocab, 1)} # Construye diccionario para mapear palabra a número entero. Los índices inician en 1.

# Conversión de cada palabra de los mensajes en índices
messages_ints = []
for message in messages_split:
  messages_ints.append([vocab_to_int[word] for word in message.split()])

In [20]:
# Cada mensaje ahora se representa como una secuencia de números (índices)
print(messages_split[0])
print(messages_ints[0])

# Cantidad de palabras únicas que contiene el diccionario
print(f"\nPalabras únicas: {len(vocab_to_int)}")

go until jurong point crazy available only in bugis n great world la e buffet cine there got amore wat
[43, 400, 3737, 709, 645, 751, 65, 8, 1104, 92, 116, 471, 1105, 142, 2509, 1106, 68, 55, 3738, 126]

Palabras únicas: 9196


### Embedding de etiquetas

Las etiquetas se convierten a números binarios para poder ser ingresados a la red neuronal. Si la etiqueta es `spam` se convierte en 1 y si la etiqueta es `ham` se convierte en 0.

In [21]:
# Se codifican las etiquetas. 1 si es 'spam', 0 si es 'ham'
labels_split = labels.split('\n')
encoded_labels = np.array([1 if label == 'spam' else 0 for label in labels_split])

### Longitud de Secuencias

Se examinan las longitudes de cada mensaje codificado para detectar mensajes de longitud cero y determinar el tamaño máximo, con el propósito de elegir adecuadamente el tamaño de padding.

In [22]:
# Cálculo de las longitudes de cada mensaje codificado
review_lens = Counter([len(x) for x in messages_ints])
print("Messages de longitud cero:", review_lens[0])
print('Máxima longitud:', max(review_lens))

Messages de longitud cero: 2
Máxima longitud: 171


Existen 2 mensajes de longitud cero, por lo que se necesita eliminarlos para poder procesarlos en la red neuronal.

In [23]:
print('Reviews antes de eliminación:', len(messages_ints))

#Extraemos los índices de todos los reviews que tienen longitud > 0
non_zero_idx = [ii for ii, review in enumerate(messages_ints) if len(review)!=0]

#Nos quedamos solo con los reviews con longitud > 0
messages_ints = [messages_ints[ii] for ii in non_zero_idx]

print('Reviews después de eliminación:', len(messages_ints))

Reviews antes de eliminación: 4968
Reviews después de eliminación: 4966


### Padding

Aplicaremos la técnica de padding a los mensajes para estandarizar la longitud de las secuencias de texto. Este paso nos permitirá convertir los datos en un array de 2D, lo cual resulta más conveniente para las etapas posteriores del procesamiento.

Tras analizar la longitud máxima de las secuencias, se determinó que estas alcanzan un máximo de 171 palabras. Para estandarizar las secuencias con padding, se optó por fijar una longitud de 30 palabras. Esta elección se basó en un proceso de ajuste experimental, donde se observó que este valor brindaba un buen desempeño en el modelo de red neuronal implementado. Por lo tanto, las secuencias que posean menos de 30 palabras se completarán con ceros, mientras que aquellas que superen esta longitud se truncarán a 30 palabras.

In [24]:
def pad_features(messages_ints, seq_length):
  """Aplica la técnica de padding a mesajes codificados para estandarizar
     la longitud de las secuencias de texto.

  Parameters:
  ----------------
  messages_ints: ndarray
            Arreglo de mensajes convertidos en enteros.

  seq_length: int
          Longitud deseada para cada secuencia de mensajes.

  Returns:
  ----------------
  features: ndarray
        Matriz 2D que contiene los mensajes con ajuste a la longitud especificada.
  """
  # Se crea una matriz de ceros con la forma correcta
  features = np.zeros((len(messages_ints), seq_length), dtype=int)

  # Se itera sobre cada mensaje en 'messages_ints' para colocarlos en la matriz con padding
  for i, row in enumerate(messages_ints):
    # Se coloca el mensaje en la fila correspondiente
    features[i, -len(row):] = np.array(row)[:seq_length]

  return features

In [25]:
# Prueba del padding

seq_length = 30 # Largo de las secuencias deseado

features = pad_features(messages_ints, seq_length=seq_length) # Aplicación del padding

# Visualización de los mensajes luego de aplicar padding
print(features.shape)
print(features[:30,:10])

(4966, 30)
[[   0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0]
 [   0    0   52  489    8   22    4  929  930    2]
 [   0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0]
 [ 752  104   68 1400   42  101  202  551   23    7]
 [   0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0   77  236   13 1401 2517 2517]
 [   0    0    0    0  710   77    4 1018  404  241]
 [   0  132   13  102 1019  753   28  117    6   85]
 [   0    0    0    0    0    0    0    0    0   21]
 [   0    0    0    0 1946 3753    2  221  180   45]
 [   0    0    0    0  187    3   16  194    4  182]
 [ 154  101 1616   12    5  153  528    2  378    3]
 [   0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    

## Partición de los datos

Se establecieron porcentajes específicos para la división de los datos procesados: el 70% para entrenamiento, el 10% para validación y el 20% para pruebas. Posteriormente, al crear los DataLoaders, se eligió un `batch_size` de 2. Esta elección se basó en encontrar un equilibrio favorable entre el rendimiento del modelo y la utilización eficiente de los recursos computacionales.

In [26]:
# Ratios de train, validation y test
train_ratio = 0.7
val_ratio = 0.1
test_ratio = 0.2

# Partición de la data en los conjuntos de train, validaion y test (messages y labels)
split_idx = int(len(features)*train_ratio)
train_x, remaining_x = features[:split_idx], features[split_idx:]
train_y, remaining_y = encoded_labels[:split_idx], encoded_labels[split_idx:]

new_ratio = val_ratio/(1-train_ratio)
test_idx = int(len(remaining_x)*new_ratio)
val_x, test_x = remaining_x[:test_idx], remaining_x[test_idx:]
val_y, test_y = remaining_y[:test_idx], remaining_y[test_idx:]

# Visualización de las dimensiones de los conjuntos y sus respectivos porcentajes con respecto
# a la data total
print("\t\t\tFeatures: \t\t\tPorcentaje:")
print(f"Train set: \t\t{train_x.shape} \t\t\t{round(100*train_x.shape[0]/features.shape[0], 1)}%",
      f"\nValidation set: \t{val_x.shape} \t\t\t{round(100*val_x.shape[0]/features.shape[0], 1)}%",
      f"\nTest set: \t\t{test_x.shape} \t\t\t{round(100*test_x.shape[0]/features.shape[0], 1)}%")

			Features: 			Porcentaje:
Train set: 		(3476, 30) 			70.0% 
Validation set: 	(496, 30) 			10.0% 
Test set: 		(994, 30) 			20.0%


In [27]:
# Creación de Tensor datasets
train_data = TensorDataset(torch.from_numpy(train_x), torch.from_numpy(train_y))
valid_data = TensorDataset(torch.from_numpy(val_x), torch.from_numpy(val_y))
test_data = TensorDataset(torch.from_numpy(test_x), torch.from_numpy(test_y))

# Creación de Dataloaders
batch_size = 2

train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
valid_loader = DataLoader(valid_data, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_data, shuffle=False, batch_size=batch_size)

## Modelo LSTM

Para poder realizar una correcta clasificación y detección de spam en mensajes de texto se optó por utilizar un modelo de red neuronal, en particular un modelo basado en una Red Neuronal Recurrente del tipo LSTM(Long Short-Term Memory). Para esto se implementó una estructura que se puede resumir en las siguientes capas:
- Una `capa embedding` que convierta tokens de palabras (índices) en embeddings de un tamaño específico.
- Una `capa LSTM` definida por el tamaño del estado oculto y el número de capas.
- Una capa de salida `fully-connected` que mapee la salida del LSTM al tamaño de salida deseado.
- Una capa de activación `sigmoide` que convierta la salida a valores [0, 1], que representan las dos clases de la data.



In [28]:
# Verificación de GPU
train_on_gpu=torch.cuda.is_available()

if(train_on_gpu):
    print('Training on GPU.')
else:
    print('No GPU available, training on CPU.')

Training on GPU.


### Creación de red neuronal

La clase `SpamRNN` es una implementación de una Red Neuronal Recurrente del tipo LSTM (Long Short-Term Memory) destinada a la clasificación de mensajes, particularmente para la identificación de spam. Al ser inicializada, establece la arquitectura de la red con capas fundamentales, incluyendo embedding, LSTM (Long Short-Term Memory), dropout para regularización y una capa lineal con función sigmoide para la salida. Los parámetros iniciales, como el tamaño del vocabulario, la dimensión de salida, el número de capas y la probabilidad de dropout, configuran la estructura de la red.

El método `forward` se encarga de realizar la propagación hacia adelante en la red, procesando los datos de entrada a través de las capas definidas. Toma los datos de entrada, calcula los embeddings correspondientes y los pasa por la capa LSTM, extrayendo luego la representación de la secuencia mediante la selección del último valor de salida. Posteriormente, realiza operaciones de dropout y una transformación lineal, finalizando con una función sigmoide para obtener las probabilidades de clase.

Por último, el método `init_hidden` inicializa el estado oculto y la memoria de la red LSTM, preparando estos componentes para su uso en la propagación hacia adelante. Este método configura los tensores necesarios para el estado oculto y la memoria de la red, estableciendo su tamaño y contenido según el tamaño del lote o batch que se esté utilizando en el entrenamiento o evaluación de la red.

In [29]:
class SpamRNN(nn.Module):
    """
    Red Neuronal Recurrente (RNN) para clasificación de mensajes de spam.

    """
    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, drop_prob=0.5):
        """Inicializa la arquitectura de la red neuronal.

        Parameters:
        -------------------
        vocab_size: int
              Tamaño del vocabulario.

        output_size:  int
              Tamaño de la salida (número de clases).

        embedding_dim: int
              Dimensión del espacio de embeddings.

        hidden_dim: int
              Dimensión del estado oculto de la red.

        n_layers: int
              Número de capas en la red neuronal.

        drop_prob: float
              Probabilidad de dropout para regularización (default=0.5).

        Returns:
        -------------------
        None

        """
        super(SpamRNN, self).__init__()

        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim

        # Capas embedding y LSTM
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers,
                            dropout=drop_prob, batch_first=True)

        # Dropout
        self.dropout = nn.Dropout(drop_prob)

        # Capa lineal y sigmoide
        self.fc = nn.Linear(hidden_dim, output_size)
        self.sig = nn.Sigmoid()

    def forward(self, x, hidden):
        """Realiza la propagación hacia adelante en la red neuronal.

        Parameters:
        -------------------
        x: torch.Tensor
        Datos de entrada (tensor con índices de palabras).

        hidden: tuple of torch.Tensor
           Estado oculto inicial.

        Returns:
        -------------------
        sig_out: torch.Tensor
            Salida de la red luego de la función sigmoide.

        hidden: tuple of torch.Tensor
            Último estado oculto.

        """
        # Obtención de los embeddings correspondientes a los datos x
        embeds = self.embedding(x)

        # Procesamiento de los embedding a través de la capa LSTM
        lstm_out, hidden = self.lstm(embeds, hidden)

        # Tomamos solo el último valor de salida del LSTM como representación de la
        # secuencia de entrada
        lstm_out = lstm_out[:,-1,:] # lstm_out no solamente me entrega la salida de la red, sino que también todos los estados ocultos de la red

        # Procesamiento por cada Dropout y fully-connected
        out = self.dropout(lstm_out)
        out = self.fc(out)

        # Procesamiento por la función sigmoide para obtener las probabilidades
        # de la clase
        sig_out = self.sig(out)

        # Retorna la salida de la función sigmoide y el último estado oculto
        return sig_out, hidden


    def init_hidden(self, batch_size):
        """Inicializa el estado oculto y la memoria de la red LSTM.

        Parameters:
        -------------------
        batch_size: int
              Tamaño del lote o batch.

        Returns:
        -------------------
        hidden: tuple of torch.Tensor
            Estado oculto y memoria inicializados.

        """
        # Crea dos nuevos tensores con tamaño n_layers x batch_size x hidden_dim,
        # inicializados a cero, para estado oculto y memoria de LSTM
        weight = next(self.parameters()).data

        if(train_on_gpu):
          hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
                   weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
        else:
          hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                   weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())

        return hidden

Para instanciar la red neuronal diseñada previamente se tuvieron que definir los parámetros de entrada los cuales definen la configuración y arquitectura específica de la red para el procesamiento de los datos. El `vocab_size`, calculado como la longitud del vocabulario más uno para incluir el padding y los tokens de palabras, determina el tamaño del espacio de embeddings y, por ende, la diversidad de palabras consideradas. El `output_size` se establece en 1, indicando que la red se usará para una tarea de clasificación binaria, la detección de spam. Los valores de `embedding_dim` y `hidden_dim`, fijados ambos en 32, definen las dimensiones del espacio de embeddings y del estado oculto de la red respectivamente, determinando la complejidad y capacidad de representación del modelo. Por último, `n_layers`, configurado en 4, especifica el número de capas que compondrán la red neuronal, influyendo en su profundidad y capacidad de aprendizaje de patrones complejos en los datos.

Se experimentó con la variación de los parámetros `embedding_dim`, `hidden_dim` y `n_layers`. La selección de estos valores se basó en buscar un equilibrio óptimo entre el rendimiento efectivo del modelo y el uso eficiente de los recursos computacionales disponibles.

In [30]:
# Instanciamos la red

# Parámetros
vocab_size = len(vocab_to_int) + 1 # +1 for zero padding + our word tokens
output_size = 1
embedding_dim = 32
hidden_dim = 32
n_layers = 4

# Creación de la red neuronal
net = SpamRNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers)
print(net)

SpamRNN(
  (embedding): Embedding(9197, 32)
  (lstm): LSTM(32, 32, num_layers=4, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=32, out_features=1, bias=True)
  (sig): Sigmoid()
)


### Entrenamiento

Para implementar el entrenamiento de la red neuronal diseñada se tuvieron que definir algunos de sus hiperparámetros los cuales desempeñan roles específicos para mejorar el aprendizaje del modelo. El `lr`, o tasa de aprendizaje, controla el tamaño de los ajustes aplicados a los pesos durante el entrenamiento, influyendo en la rapidez con la que la red converge hacia soluciones óptimas. La `criterion`, que representa la función de pérdida, evalúa la diferencia entre las predicciones del modelo y los valores reales, proporcionando una medida de cuán precisa es la red. Finalmente, el `optimizer`, configurado como el optimizador Adam, se encarga de ajustar los pesos de la red en función de la pérdida calculada, utilizando la tasa de aprendizaje establecida para optimizar la convergencia hacia soluciones más precisas y óptimas.

In [31]:
# Hiperparámetros del entrenamiento de la red neuronal
lr=0.001 # Learning rate
criterion = nn.BCELoss() # Función de pérdidas
optimizer = torch.optim.Adam(net.parameters(), lr=lr) # Método de optimización

In [32]:
# Parámetros de entrenamiento
epochs = 4

counter = 0
print_every = 100
clip=5 # Gradient clipping

# Enviar red al GPU
if(train_on_gpu):
    net.cuda()

# Inicio de entrenamiento
net.train()
# Bucle de entrenamiento
for e in range(epochs):
    # Inicializar estado oculto
    h = net.init_hidden(batch_size)

    # Bucle para batches
    for inputs, labels in train_loader:
        counter += 1

        if(train_on_gpu):
            inputs, labels = inputs.cuda(), labels.cuda()

        # Crear nuevas variables para estados ocultos, sino se haría
        # backprop para todos los pasos del bucle
        h = tuple([each.data for each in h])

        net.zero_grad()

        # Hacer pasada forward
        output, h = net(inputs, h)

        # Calcular loss y hacer backprop
        loss = criterion(output.squeeze(), labels.float())
        loss.backward()
        # gradient clipping
        nn.utils.clip_grad_norm_(net.parameters(), clip)
        optimizer.step()

        # Mensajes
        if counter % print_every == 0:
            # Validation loss
            val_h = net.init_hidden(batch_size)
            val_losses = []
            net.eval()
            for inputs, labels in valid_loader:

                val_h = tuple([each.data for each in val_h])

                if(train_on_gpu):
                    inputs, labels = inputs.cuda(), labels.cuda()

                output, val_h = net(inputs, val_h)
                val_loss = criterion(output.squeeze(), labels.float())

                val_losses.append(val_loss.item())

            net.train()
            print("Época: {}/{}...".format(e+1, epochs),
                  "Paso: {}...".format(counter),
                  "Loss: {:.6f}...".format(loss.item()),
                  "Val Loss: {:.6f}".format(np.mean(val_losses)))

Época: 1/4... Paso: 100... Loss: 0.202755... Val Loss: 0.413770
Época: 1/4... Paso: 200... Loss: 0.214990... Val Loss: 0.407546
Época: 1/4... Paso: 300... Loss: 0.985260... Val Loss: 0.407120
Época: 1/4... Paso: 400... Loss: 0.164052... Val Loss: 0.407018
Época: 1/4... Paso: 500... Loss: 0.131785... Val Loss: 0.407254
Época: 1/4... Paso: 600... Loss: 0.128321... Val Loss: 0.408077
Época: 1/4... Paso: 700... Loss: 0.165827... Val Loss: 0.409657
Época: 1/4... Paso: 800... Loss: 0.213870... Val Loss: 0.407896
Época: 1/4... Paso: 900... Loss: 0.147851... Val Loss: 0.415872
Época: 1/4... Paso: 1000... Loss: 0.201805... Val Loss: 0.407661
Época: 1/4... Paso: 1100... Loss: 0.212983... Val Loss: 0.407572
Época: 1/4... Paso: 1200... Loss: 0.124233... Val Loss: 0.407108
Época: 1/4... Paso: 1300... Loss: 0.166929... Val Loss: 0.407249
Época: 1/4... Paso: 1400... Loss: 0.299484... Val Loss: 0.407505
Época: 1/4... Paso: 1500... Loss: 0.181145... Val Loss: 0.407125
Época: 1/4... Paso: 1600... Loss: 

El entrenamiento del modelo se ejecutó a lo largo de 4 épocas, culminando con una pérdida de entrenamiento cercana a 0.11 y una pérdida de validación de alrededor de 0.43. Aunque estos valores son considerados aceptables en el proceso de entrenamiento, no son concluyentes por sí solos para evaluar el rendimiento general del modelo. La verdadera evaluación de su capacidad se debe realizar en la data de test, permitiendo determinar la capacidad de aprendizaje y generalización del modelo sobre datos no vistos previamente. Los resultados obtenidos en la data de test proporcionarán una comprensión más clara y definitiva sobre el desempeño y la capacidad de generalización del modelo entrenado.

### Testing

In [33]:
# Calculo del accuracy de test

test_losses = [] # track loss
num_correct = 0

# Iniciar estado oculto
h = net.init_hidden(batch_size)

net.eval()
for inputs, labels in test_loader:

    h = tuple([each.data for each in h])

    if(train_on_gpu):
        inputs, labels = inputs.cuda(), labels.cuda()

    output, h = net(inputs, h)

    test_loss = criterion(output.squeeze(), labels.float())
    test_losses.append(test_loss.item())

    # Convertir probabilidades a clases (0,1)
    pred = torch.round(output.squeeze())

    # Comparar predicciones a labels
    correct_tensor = pred.eq(labels.float().view_as(pred))
    correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
    num_correct += np.sum(correct)


# -- stats! -- ##
# avg test loss
print("Test loss: {:.3f}".format(np.mean(test_losses)))

# Accuracy de test
test_acc = num_correct/len(test_loader.dataset)
print("Test accuracy: {:.3f}".format(test_acc))

Test loss: 0.374
Test accuracy: 0.886


Después de analizar exhaustivamente los resultados del entrenamiento y test del modelo, se ha podido evaluar su rendimiento a través de diversas métricas. Las pérdidas registradas durante el entrenamiento (0.11) y la validación (0.43) apuntan a un aprendizaje efectivo del modelo en esta fase, aunque la diferencia entre estas pérdidas sugiere una posible presencia de sobreajuste. Sin embargo, la pérdida de test (0.37) se mantiene cercana a la de validación, lo que sugiere que el modelo generaliza bien sobre datos no vistos, lo cual es confirmado por el accuracy de test del 88%. Esto indica que el modelo es capaz de predecir correctamente alrededor del 88% de las muestras en el conjunto de test, lo que es un buen indicador de su capacidad predictiva y generalización sobre datos nuevos.